# RAG Haystack Demo
**Demo for Iteration 2 of the prototype.**

Installs:\
!pip install haystack-ai\
!pip install python-pptx\
!pip install python-docx\
!pip install pypdf2\
!pip install trafilatura\
!pip install unstructured-client\
!pip install unstructured-fileconverter-haystack\
!pip install unstructured\
!pip install sentence-transformers

In [ ]:
from haystack import Pipeline
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.converters.unstructured import UnstructuredFileConverter
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.utils import Secret
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever, InMemoryEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.joiners.document_joiner import DocumentJoiner
from haystack.components.builders import PromptBuilder

In [ ]:
import os

# Groq API Key
GROQ_OPENAI_API_KEY = userdata.get('GROQ_OPENAI_API_KEY')
os.environ['GROQ_OPENAI_API_KEY'] = GROQ_OPENAI_API_KEY

# Unstructured API Key
UNSTRUCTURED_API_KEY = userdata.get('UNSTRUCTURED_API_KEY')
os.environ['UNSTRUCTURED_API_KEY'] = UNSTRUCTURED_API_KEY

# **Indexing**

In [ ]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

### Converter

In [ ]:
converter = UnstructuredFileConverter()
result = converter.run(paths=["demo_guide.pdf"])
print(result['documents'][0].content)

Converting files to Haystack Documents: 1it [00:03,  3.99s/it]

Outdated User Guide (Dummy Data)

1. Access the System: Open your web browser and navigate to

www.oldcompanyportal.com to access the internal project management system. 2. Log In: Enter your username and temporary password sent to you via email. Click

"Login" to access the system.

3. Change Password: Go to the "Account Settings" section and change your temporary password to a new, secure one.

4. Complete Profile Information: Fill out your profile with required information, including job title, department, and contact information.

5. Join Your Team’s Project: Locate the “Projects” tab, find your team’s project, and click “Join” to get access.

6. Set Up Notifications: Configure your notification preferences under “Settings” to receive updates about project activities and deadlines.

7. Review Training Materials: Download and review the training materials available in the “Resources” section to familiarize yourself with the system’s features.

8. Attend Orientation Webinar: Join the

### Cleaner
Not used in this demo.

In [ ]:
# from haystack.components.preprocessors import DocumentCleaner

# cleaner = DocumentCleaner(
#   ascii_only=True,
# 	remove_empty_lines=True,
# 	remove_extra_whitespaces=True,
# 	remove_repeated_substrings=False)

# cleaned_result = cleaner.run(documents=result['documents'])
# print(cleaned_result['documents'][0].content)

### Splitter/Chunker

In [ ]:
splitter = DocumentSplitter(split_by="passage", split_length=1, split_overlap=0)
split_result = splitter.run(documents=result['documents'])

for document in split_result["documents"]:
    print(f"{document.content}\n")

Outdated User Guide (Dummy Data)



1. Access the System: Open your web browser and navigate to



www.oldcompanyportal.com to access the internal project management system. 2. Log In: Enter your username and temporary password sent to you via email. Click



"Login" to access the system.



3. Change Password: Go to the "Account Settings" section and change your temporary password to a new, secure one.



4. Complete Profile Information: Fill out your profile with required information, including job title, department, and contact information.



5. Join Your Team’s Project: Locate the “Projects” tab, find your team’s project, and click “Join” to get access.



6. Set Up Notifications: Configure your notification preferences under “Settings” to receive updates about project activities and deadlines.



7. Review Training Materials: Download and review the training materials available in the “Resources” section to familiarize yourself with the system’s features.



8. Attend Orientation

# Embedder

In [ ]:
embedder = SentenceTransformersDocumentEmbedder()
embedder.warm_up()

result = embedder.run(split_result['documents'])
print(result['documents'][0].embedding)

# [-0.07804739475250244, 0.1498992145061493, ...]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.03033415786921978, 0.05205836892127991, -0.014277526177465916, -0.00599376717582345, 0.016766708344221115, 0.03268156200647354, 0.032735712826251984, 0.05453116074204445, -0.01761702075600624, -0.01664125546813011, 0.019778572022914886, -0.02227970026433468, 0.05907652899622917, 0.006886268500238657, -0.005678690969944, 0.015786822885274887, 0.019832633435726166, 0.021426865831017494, -0.04754222184419632, -0.01627623662352562, -0.013668759725987911, 0.014014596119523048, 0.0044463821686804295, 5.681907350663096e-05, -0.010881326161324978, 0.013826396316289902, -0.0050155362114310265, 0.027846809476614, -0.05109275504946709, -0.07425713539123535, 0.05998779460787773, 0.02396077662706375, 0.0355590395629406, -0.026728417724370956, 1.4078402728046058e-06, -0.05198126658797264, -0.0002693743444979191, 0.019516142085194588, -0.09193960577249527, 0.03845549374818802, 0.034137070178985596, -0.0034547424875199795, 0.01892569102346897, 0.016434265300631523, 0.004379418212920427, 0.000275478

# Writer (Load Embeddings)

In [ ]:
document_writer = DocumentWriter(document_store = document_store)
document_writer.run(documents=result['documents'])

{'documents_written': 10}

# **Querying**

In [ ]:
# Query and Query embeddings
text_embedder = SentenceTransformersTextEmbedder()
text_embedder.warm_up()

query="How do I change my password"
query_embedding=text_embedder.run(query)

print(query)
print(query_embedding['embedding'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

How do I change my password
[-0.025806577876210213, -0.02617955021560192, 0.013890509493649006, 0.060708411037921906, -0.0042159948498010635, 0.003371739061549306, -0.00877077504992485, -0.010790486820042133, 0.011666130274534225, -0.014651144854724407, -0.025269413366913795, -0.03953014686703682, 0.08010268956422806, -0.011932416819036007, 0.028325509279966354, 0.019653167575597763, -0.03136264905333519, -0.04445323720574379, 0.0028921181801706553, -0.007115676533430815, 0.0024020096752792597, -0.0022795761469751596, -0.0428677462041378, 8.681674444233067e-06, -0.009755945764482021, 0.0006160380435176194, 0.039631132036447525, -0.006261938717216253, 0.06090069189667702, 0.015706704929471016, -0.0015388855244964361, 0.022637177258729935, -0.009640956297516823, 0.009830606169998646, 1.1718690302586765e-06, -0.027188662439584732, -0.01784571260213852, -0.0016574900364503264, 0.0037313136272132397, 0.023208845406770706, 0.0015517588471993804, -0.013476280495524406, 0.0004424373328220099, 

### Retriever

In [ ]:
# Vector Search
embedding_retriever = InMemoryEmbeddingRetriever(document_store)

# Keyword Search
bm25_retriever = InMemoryBM25Retriever(document_store)

In [ ]:
retrieved_result=embedding_retriever.run(query_embedding['embedding'])
print(retrieved_result['documents'][0].content)

retrieved_result=bm25_retriever.run(query)
print(retrieved_result['documents'][0].content)

3. Change Password: Go to the "Account Settings" section and change your temporary password to a new, secure one.


3. Change Password: Go to the "Account Settings" section and change your temporary password to a new, secure one.




### Document Joiner (For Hybrid Retrieval)
Not used in this demo.

In [ ]:
# from haystack.components.joiners import DocumentJoiner

# document_joiner = DocumentJoiner()

### Prompt Builder

In [ ]:
prompt_template = "Answer the query '{{ query }}' using the following contextContext: {{ context }}; Answer:"
builder = PromptBuilder(template=prompt_template)
input=builder.run(query=query, context=retrieved_result['documents'][0].content)
print(input)

{'prompt': 'Answer the query \'How do I change my password\' using the following contextContext: 3. Change Password: Go to the "Account Settings" section and change your temporary password to a new, secure one.\n\n; Answer:'}


In [ ]:
llm = OpenAIGenerator(
        api_key=Secret.from_env_var("GROQ_OPENAI_API_KEY"),
        api_base_url="https://api.groq.com/openai/v1",
        model="llama3-8b-8192",
        generation_kwargs={"temperature": 0}
    )

In [ ]:
response=llm.run(input['prompt'])
print(response['replies'][0])

To change your password, follow these steps:

Go to the "Account Settings" section.


# RAG with 2 documents


1.   Outdated Guide
2.   Reference Material



In [ ]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
indexing_pipeline=Pipeline()
indexing_pipeline.add_component("converter", UnstructuredFileConverter())
indexing_pipeline.add_component("splitter", DocumentSplitter(split_by="passage", split_length=1, split_overlap=0))
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder())
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.connect("converter", "splitter")
indexing_pipeline.connect("splitter", "embedder")
indexing_pipeline.connect("embedder", "writer")
indexing_pipeline.run({"converter": {"paths": ["demo_guide.pdf", "demo_reference.pdf"]}})

query_pipeline=Pipeline()
query_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder())
query_pipeline.add_component("embedding_retriever", InMemoryEmbeddingRetriever(document_store))
query_pipeline.add_component("bm25_retriever", InMemoryBM25Retriever(document_store))
query_pipeline.add_component("document_joiner", DocumentJoiner(join_mode="merge"))
query_pipeline.connect("text_embedder", "embedding_retriever")
query_pipeline.connect("bm25_retriever", "document_joiner")
query_pipeline.connect("embedding_retriever", "document_joiner")

query = "How do I access the project management system"

result = query_pipeline.run(
    {"text_embedder": {"text": query}, "bm25_retriever": {"query": query}}
)

Converting files to Haystack Documents: 2it [00:06,  3.15s/it]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The following context is retrieved from the query: "How do I access the project management system"

In [ ]:
print(result['document_joiner']['documents'][0].content)
print(result['document_joiner']['documents'][1].content)
print(result['document_joiner']['documents'][2].content)

• Access the New Portal: Open your web browser and navigate to www.newcompanyportal.com to access the updated project management system.


www.oldcompanyportal.com to access the internal project management system. 2. Log In: Enter your username and temporary password sent to you via email. Click


"Login" to access the system.


